In [1]:
from common import *

import xml.etree.ElementTree as ET
from pymongo import MongoClient
import pymongo

from pymystem3 import Mystem

Using TensorFlow backend.


In [2]:
fname = join(join(DATA_FOLDER, 'rawdata'), 'all_docs_raw.txt')

In [3]:
# all_docs = glob(join(DATA_FOLDER, 'rawdata') + '/**/*.xml', recursive=True)
# all_docs = sorted(all_docs)
# with open(fname, mode='wt', encoding='utf-8') as f:
#     f.write('\n'.join(all_docs))

In [4]:
with open(fname, mode='r', encoding='utf-8') as f:
    all_docs = f.read().splitlines()

In [5]:
len(all_docs)

1266990

In [6]:
def get_node(root, name, lang='ru'):
    for node in root.iter(name):
        if 'lang' in node.attrib:
            if node.attrib['lang'].lower() == lang:
                return node
        else:
            return node

In [2]:
def parse_xml(fname, mstem):
    doc = {}

    with open(fname, 'r') as f:
        text = f.read()
        text = text.replace('&deg', 'deg')
    
    try:
        root = ET.fromstring(text)
    except:
        print(fname)
        return
    
    doc.update(root.attrib)

    SDOBI = get_node(root, 'SDOBI')
    if SDOBI is not None:
        node = SDOBI.find('./B500/ru-b540/ru-b542')
        if node is not None:
            name = node.text.lower()
            doc['name_raw'] = name
            doc['name'] = mstem.lemmatize(name)
        else:
            print(fname)            
    else:
        print('no SDOBI: ' + fn)

    node = get_node(root, 'abstract')
    if node is not None:
        doc['abstract_raw'] = ' '.join(node.itertext()).lower()
        doc['abstract'] = mstem.lemmatize(doc['abstract_raw'])

    if SDOBI is not None:
        node = SDOBI.find('./B500/B560/ru-b560')
        if node is not None:
            doc['ru-b560'] = ' '.join(node.itertext()).lower()

    node = get_node(root, 'description')
    if node is not None:
        doc['description_raw'] = ' '.join(node.itertext()).lower()
        doc['description'] = mstem.lemmatize(doc['description_raw'])

    node = get_node(root, 'claims')
    if node is not None:
        doc['claims_raw'] = ' '.join(node.itertext()).lower()
        doc['claims'] = mstem.lemmatize(doc['claims_raw'])
            
    return doc

# Test

In [329]:
afile = '../data/rawdata/2016/RU/A/20161227/2015116494/2015116494.xml'
c1file = '../data/rawdata/2016/RU/C1/20161227/0002605713/0002605713.xml'
c2file = '../data/rawdata/2016/RU/C2/20160127/0002573901/0002573901.xml'
ufile = '../data/rawdata/2016/RU/U1/20161210/0000166582/0000166582.xml'

In [381]:
doc = parse_xml(c2file)

In [346]:
print(doc['description_raw'])

предлагаемое техническое решение относится к устройствам для уплотнения валов, штоков, запорной, арматуры, люков, фланцевых соединений и реакторов, в том числе при работе в условиях повышенных температур, и может найти применение в химической, нефтехимической, машиностроительной, энергетической, атомной, судостроительной и других отраслях промышленности. известно уплотнительное устройство, содержащее корпус, в котором выполнена камера, имеющая форму цилиндра с переходом в форму эллипсоида. в камеру размещена уплотнительная набивка и шток. набивка представляет собой спираль из фторопласта, между витками которой расположена пластинчатая набивка, обеспечивающая герметичность в период эксплуатации (описание изобретения к патенту рф №2282084, f16j 15/18, 2006). к причинам, препятствующим достижению заданного технического результата, относится износ трущихся поверхностей пластинчатой набивки о стенки штока, особенно в условиях повышенной температуры, что приводит к потере герметичности и сни

In [349]:
doc

{'abstract_raw': 'изобретение относится к способу получения поверхностного покрытия, содержащему этапы обеспечения слоя износа на основе пвх (1), по существу, свободного от неорганических наполнителей, нанесения на упомянутый слой на основе пвх (1) композиции верхнего покрытия, содержащей поливиниловый спирт (пвс) и соединение силана, содержащее по меньшей мере одну аминогруппу, нагревание упомянутого слоя на основе пвх (1) и упомянутой композиции верхнего покрытия при температуре, эквивалентной или превосходящей 150°c, для образования верхнего покрытия слоя износа (2). результатом является получение поверхностного покрытия, сочетающего свойства физической устойчивости и свойства химического и газового барьера. 2 н. и 12 з.п. ф-лы, 4 ил., 6 табл.',
 'bullid': 'd03',
 'claims_raw': '1. способ получения поверхностного покрытия на основе поливинилхлорида (пвх), включающий следующие этапы: - обеспечение слоя износа на основе пвх (1), причем количество содержащихся неорганических наполнител

# Fill database with data

In [8]:
client = MongoClient()
db = client.fips

In [9]:
patents = db.patents
result = db.patents.create_index([('doc-number', pymongo.ASCENDING),
                                   ('kind', pymongo.ASCENDING),
                                   ('country', pymongo.ASCENDING),
                                   ('date-publ', pymongo.ASCENDING)], unique=True)

In [10]:
sorted(list(db.patents.index_information()))

['_id_', 'doc-number_1_kind_1_country_1_date-publ_1']

In [11]:
docs = []
for fn in tqdm(all_docs):
    doc = parse_xml(fn)
    if doc is not None:
        patents.insert_one(doc)
#         docs.append(doc)

 22%|██▏       | 272488/1266990 [05:47<47:47, 346.84it/s]  

../data/rawdata/1999/RU/C1/19990220/0002126409/0002126409.xml


 51%|█████     | 645208/1266990 [16:31<22:21, 463.55it/s]  

KeyboardInterrupt: 

In [ ]:
# patents.insert_many(docs)

In [475]:
for d in patents.find():
    print(d)